In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
camvid_path = kagglehub.dataset_download('carlolepelaars/camvid')

print('Data source import complete.')


# **Programming Assignment 1 - Semantic Segmentation**

**Professor**: Dário Oliveira  
**Monitor**: Lívia Meinhardt

Neste trabalho prático, vocês irão investigar o desempenho de modelos de segmentação semântica, utilizando redes U-Net e DeepLabV3. A proposta vai além de treinar modelos: vocês deverão analisar onde eles funcionam bem (ou não), interpretar os erros e justificar suas decisões com base nos conceitos vistos em aula.


# **Instruções:**

1. **Escolha do Ambiente de Execução**:  
   Utilize Google Colab ou Kaggle Notebook. Recomendamos iniciar seu notebook no Kaggle diretamente da página do dataset [CamVid](https://www.kaggle.com/datasets/carlolepelaars/camvid).

3. **Criação de um Dataset Customizado**:  
   As máscaras vêm em formato RGB. Use o CSV fornecido para converter as cores em labels (classe por pixel). Crie seu próprio Dataset em PyTorch.

4. **Construção da Arquitetura U-Net**:  
   Implemente uma U-Net (ou ResUNet), com a possibilidade de variar sua profundidade (número de blocos codificadores/decodificadores). Explore como isso impacta o desempenho e a quantidade de parâmetros.

   <div>
   <img src="https://camo.githubusercontent.com/6b548ee09b97874014d72903c891360beb0989e74b4585249436421558faa89d/68747470733a2f2f692e696d6775722e636f6d2f6a6544567071462e706e67" width=600>
   </div>

5. **Função de Treinamento**:  
   Crie uma função de treinamento e registre métricas (loss, acurácia) em cada época. A cada 5 ou 10 épocas, visualize uma predição (imagem original, máscara verdadeira e predita).

6. **Experimentação**:
    Além da profundidade da rede e como ela afeta desempenho e quantidade de parâmetros, explore pelo menos dois otimizadores e funções de perda adequadas para segmentação semântica. Fundamente as escolhas e explique os resultados de acordo com a teoria vista em aula.

8. **Avaliação**:
   Implemente as métricas por classe: Precisão e IoU. Além da média geral, use para identificar as classes com pior desempenho e caracterizar seu melhor modelo.

9. **Explicabilidade com Mapas de Erro**:
   Escolha imagens com desempenho ruim (menor IoU ou precisão) e gere mapas de erro. Analise visualmente onde o modelo erra (bordas, classes confundidas, objetos pequenos, etc.). Relacione os erros às métricas.

10. **Data Augmentation**:  
     Implemente alguma forma de data augmentation. Avalie se houve ganho em desempenho. Justifique.

11. **Fine-Tuning do DeepLabV3**:
   Realize o fine-tuning do modelo [DeepLabV3](https://docs.pytorch.org/vision/main/models/generated/torchvision.models.segmentation.deeplabv3_resnet50.html#torchvision.models.segmentation.deeplabv3_resnet50) pré-treinado ajustando `classifier` e `aux_classifier` para o número de classes do seu dataset. Congele o backbone conforme necessário e treine o modelo. Compare seu desempenho com a melhor U-Net em termos de métricas, número de parâmetros e qualidade visual das predições.

12. **Apresentação Final**:  
    Ao final, prepare uma apresentação resumindo os passos seguidos, resultados obtidos, gráficos de perdas e acurácia, e discussões sobre o desempenho do modelo. Lembre de fundamentar a discussão com os aspectos teoricos vistos em sala de aula.


### **Pontos Importantes:**

- Escolher adequadamente o tamanho do BATCH, Loss Function e Otimizador e saber o motivo de cada escolha;
- Monitore o uso das GPUs, o kaggle te informa quantidade de tempo disponível, mas o colab não;
- Observe as classes com mais erros por parte do modelo;
- Adicione gráficos de perda e acurácia na sua apresentação;
- Coloque imagens das predições do modelo;
- Use **Pytorch!!!**.

Note que as instruções acima são os requisitos da entrega, mas não precisam ser feitas exatamente nesta ordem. Você pode implementar o lógica de validação do modelo, testar uma versão inicial e modificar conforme os resultados obtidos. Lembre-se de fundamentas suas escolhas e fluxo de trabalho na apresentação.

In [ ]:
import torch
from torch import nn
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR

from torchvision.transforms import ToTensor, Lambda
from torchvision.io import decode_image
import torchvision.transforms as T
from torchvision.models.segmentation import deeplabv3_resnet50, DeepLabV3_ResNet50_Weights
from torchvision.models.segmentation.deeplabv3 import DeepLabHead, FCNHead

from sklearn.metrics import confusion_matrix

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

from tqdm import tqdm

import os
import glob

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"using device '{device}'")

In [ ]:
#----------------------------I/O no Kaggle------------------------------
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        continue
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#------------------Dataset para carregar as imagens---------------------
class CamVidDataset(Dataset):

    #inicializa o dataset
    def __init__(self, type_, img_dim = [720, 960], dataset_path = ("/", "kaggle", "input", "camvid"), device = "cpu"):
        #Carrega o caminho do conjunto desejado (val, train, test)
        self.base_path_data = os.path.join(*dataset_path, "CamVid", f"{type_}")
        self.base_path_labels = os.path.join(*dataset_path, "CamVid", f"{type_}_labels")
        self.data = glob.glob(os.path.join(self.base_path_data, "*.png"))

        #Carrega o csv com as classes e trata ele
        df = pd.read_csv(os.path.join(*dataset_path, "CamVid", "class_dict.csv"))
        df = df.reset_index().rename(columns = {"index": "class"})
        self.df = df.set_index(["r", "g", "b"])

        #Faz uma array de lookup pra converter o rgb pra classe mais eficientemente
        self.lookup_classes = np.zeros((256, 256, 256), dtype = "uint8")
        for (r, g, b), row in self.df.iterrows():
            self.lookup_classes[r, g, b] = row["class"]

        self.device = device

        self.resize_img = T.Resize(size=img_dim, interpolation=T.InterpolationMode.BILINEAR, antialias=True)
        self.resize_label = T.Resize(size=img_dim, interpolation=T.InterpolationMode.NEAREST)
        self.normalize_img = T.Normalize([105.3549, 107.8312, 109.6686], [5637.0488**0.5, 5869.9844**0.5, 5731.8906**0.5])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        #Pega o path da imagem e da label e carrega
        img_path = self.data[idx]
        name = img_path.split("/")[-1].split(".")[0]
        label_path = os.path.join(self.base_path_labels, f"{name}_L.png")

        image = decode_image(img_path).to(torch.float).to(self.device)
        label = decode_image(label_path)

        image = self.resize_img(image)
        image = self.normalize_img(image)
        label = self.resize_label(label)

        #Procura na array de lookup as classes de cada pixel em um acesso só
        #(mais eficiente)
        label_permute = label.permute(1, 2, 0)
        r_channel = label_permute[:, :, 0]
        g_channel = label_permute[:, :, 1]
        b_channel = label_permute[:, :, 2]
        label = torch.Tensor(self.lookup_classes[r_channel, g_channel, b_channel]).to(self.device)

        return image, label

    def orig_image(self, idx):
        img_path = self.data[idx]
        return decode_image(img_path)

train_data = CamVidDataset("train", img_dim=[256, 256], dataset_path = [camvid_path], device = device)
val_data = CamVidDataset("val", img_dim=[256, 256], dataset_path = [camvid_path], device = device)
test_data = CamVidDataset("test", img_dim=[256, 256], dataset_path = [camvid_path], device = device)
test_data[0][0].dtype, test_data[0][1].dtype

In [ ]:
#---------------Calculo a frequência das classes--------------------------
classes = pd.read_csv(os.path.join(camvid_path, "CamVid", "class_dict.csv"))
colors = torch.Tensor(classes[["r", "g", "b"]].to_numpy()).to(int).to(device)

freqs = torch.zeros((32,)).to(int).to(device)
for image, label in train_data:
    class_, count = label.unique(return_counts = True)
    freqs[class_.to(int)] += count

classes["frequencies"] = freqs.cpu()
classes

In [ ]:
#---------------------------UNet incial----------------------------------
class ConvBlock(nn.Module):
    def __init__(self, index = 1):
        super().__init__()

        self.index = index
        out_channels = 2 ** (5 + index)
        in_channels = out_channels//2 if index != 1 else 3

        self.conv_1 = nn.Conv2d(in_channels = in_channels, out_channels = out_channels, kernel_size = 3, padding = "same")
        self.conv_2 = nn.Conv2d(in_channels = out_channels, out_channels = out_channels, kernel_size = 3, padding = "same")
        self.max_pool = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv_1(x))
        x = self.relu(self.conv_2(x))
        y = self.max_pool(x)
        return x, y

class UpBlock(nn.Module):
    def __init__(self, index = 1):
        super().__init__()

        self.index = index
        in_channels = 2 ** (6 + index)
        out_channels = in_channels//2

        self.transpose = nn.ConvTranspose2d(in_channels = in_channels, out_channels = out_channels, kernel_size = 2, stride = 2)
        self.conv_1 = nn.Conv2d(in_channels = in_channels, out_channels = out_channels, kernel_size = 3, padding = "same")
        self.conv_2 = nn.Conv2d(in_channels = out_channels, out_channels = out_channels, kernel_size = 3, padding = "same")
        self.relu = nn.ReLU()

    def forward(self, x, skip):
        x = self.transpose(x)
        cat_dim = len(x.shape) - 3
        x = torch.cat((skip, x), dim = cat_dim)
        x = self.relu(self.conv_1(x))
        x = self.relu(self.conv_2(x))
        return x

class UNet(nn.Module):
    def __init__(self, depth = 4):
        super().__init__()

        self.convs = nn.ModuleList()
        for i in range(depth):
            self.convs.append(ConvBlock(i + 1).to(device))

        in_channels = 2 ** (5 + depth)
        out_channels = in_channels * 2
        self.bottleneck = nn.Sequential(
            nn.Conv2d(in_channels = in_channels, out_channels = out_channels, kernel_size = 3, padding = "same"), nn.ReLU(),
            nn.Conv2d(in_channels = out_channels, out_channels = out_channels, kernel_size = 3, padding = "same"), nn.ReLU()
        )

        self.ups = nn.ModuleList()
        for i in range(depth):
            self.ups.insert(0, UpBlock(i + 1).to(device))

        self.classifier = nn.Conv2d(in_channels = 64, out_channels = 32, kernel_size = 1, padding = "same")

    def forward(self, x):
        skip_connections = []
        for block in self.convs:
            conv_out, x = block(x)
            skip_connections.insert(0, conv_out)

        x = self.bottleneck(x)

        for i, block in enumerate(self.ups):
            x = block(x, skip_connections[i])

        logits = self.classifier(x)

        return logits

In [ ]:
#-------------------------Inferência básica----------------------------------
base_model = UNet().to(device)
pred = base_model(test_data[0][0])
pred_softmax = pred.argmax(axis = 0)
print(pred.shape)

In [ ]:
#---------------------Plot de imagens----------------------------
def display_image(model, dataset, index, deeplab = False):
    with torch.no_grad():
        if not deeplab:
            pred = model(dataset[index][0])
        else:
            pred = model(dataset[index][0].unsqueeze(0))[0]
        pred_softmax = pred.argmax(axis = 0)


    fig, axs = plt.subplots(1, 3, figsize = (10, 24))

    axs[0].imshow(dataset.orig_image(index).permute(1, 2, 0).to(int).cpu())
    axs[0].set_title("Original image")

    axs[1].imshow(colors[pred_softmax].cpu())
    axs[1].set_title("Prediction")

    axs[2].imshow(colors[dataset[index][1].to(int)].to(int).cpu())
    axs[2].set_title("Original label")
    #train_data[0][1].unique(return_counts = True)

    plt.show()

display_image(model_4, test_data, 0)

In [ ]:
#--------------------Loops de treino e teste-------------
def loop_treino(train_dataloader, val_dataloader, modelo, loss_fc, otimizador, scheduler, epochs = 30):
    modelo.train()
    losses = {
        "train": {"acc": [], "loss": []},
        "val": {"acc": [], "loss": []}
    }

    for epoch in range(epochs):
        #------Treino----
        running_loss_train, correct_predictions_train, total_samples = 0.0, 0, 0
        progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1} | train")
        for X, y in progress_bar:
            pred = modelo(X)
            loss = loss_fc(pred, y.to(int))

            otimizador.zero_grad()
            loss.backward()
            otimizador.step()

            running_loss_train += loss.item() * X.size(0)
            _, predicted_labels = torch.max(pred, 1)
            correct_predictions_train += (predicted_labels == y).sum().item()
            total_samples += torch.numel(y)

            progress_bar.set_postfix(loss=loss.item())

        epoch_loss_train = running_loss_train / len(train_dataloader.dataset)
        epoch_acc_train = correct_predictions_train / total_samples
        #-----Validação----
        running_loss_val, correct_predictions_val, total_samples = 0.0, 0, 0
        progress_bar = tqdm(val_dataloader, desc=f"Epoch {epoch+1} | val")

        with torch.no_grad():
            for X, y in progress_bar:
                pred = modelo(X)
                loss = loss_fc(pred, y.to(int))

                running_loss_val += loss.item() * X.size(0)
                _, predicted_labels = torch.max(pred, 1)
                correct_predictions_val += (predicted_labels == y).sum().item()
                total_samples += torch.numel(y)

                progress_bar.set_postfix(loss=loss.item())

        epoch_loss_val = running_loss_val / len(val_dataloader.dataset)
        epoch_acc_val = correct_predictions_val / total_samples
        #-----Display----
        scheduler.step()
        print(f"Fim Epoch {epoch+1}:")
        print(f"   -> Loss train: {epoch_loss_train:.6f} | Loss val: {epoch_loss_val:.6f}")
        print(f"   -> Acc train: {epoch_acc_train:.6f}  | Acc val: {epoch_acc_val:.6f}")
        print(f"   -> LR: {scheduler.get_last_lr()[0]:.6f}\n")

        display_image(modelo, val_dataloader.dataset, 0)

        losses["train"]["acc"].append(epoch_acc_train)
        losses["train"]["loss"].append(epoch_loss_train)
        losses["val"]["acc"].append(epoch_acc_val)
        losses["val"]["loss"].append(epoch_loss_val)
        
    return losses


def loop_teste(dataloader, modelo, loss_fc):
    modelo.eval()

    tamanho = len(dataloader.dataset)
    num_batches = len(dataloader)

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = modelo(X)
            loss_test += loss_fc(pred, y).item()
            acertos += (pred.argmax(1) == y).type(torch.float).sum().item()

    loss_test /= num_batches
    print(f"Erro no Teste: \n Perda média: {loss_test:>8f} \n")

In [ ]:
#--------------Data loader, erro, otimizador, etc----------------------
batch_size = 12
train_dataloader = DataLoader(train_data, batch_size = batch_size, shuffle = True)
val_dataloader = DataLoader(val_data, batch_size = batch_size, shuffle = True)

In [ ]:
#--------------Focal Loss-------------------
class FocalLoss(nn.Module):
    def __init__(self, gamma = 2, weight = None) -> None:
        self.gamma = gamma
        self.weight = weight

    def __call__(self, inputs, targets):
        ce_loss = torch.nn.functional.cross_entropy(inputs, targets, reduction='none', weight = self.weight)
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss
        return focal_loss.mean()

In [ ]:
#----------------UNet 4 layers CE Loss NAdam-------------------
model_4_ce = UNet(4).to(device)
optimizer = torch.optim.NAdam(model_4_ce.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=15, gamma=0.1)

weights = classes["frequencies"].sum()/(classes["frequencies"] + 1e0)
error_ce = nn.CrossEntropyLoss()#weight = torch.Tensor(weights).to(device))

In [ ]:
historico = loop_treino(train_dataloader, val_dataloader, model_4_ce, error_ce, optimizer, scheduler)

In [ ]:
torch.save(model_4_cd.state_dict(), "unet_4_ce_loss_nadam.pth")

# Avaliação

In [ ]:
def model_eval(dataloader, model, num_classes, class_names):
    model.to(device)
    model.eval()

    preds = []
    labels = []

    image_ious = []

    full_cm = np.zeros((num_classes, num_classes))

    progress_bar = tqdm(dataloader, desc="Evaluating")
    with torch.no_grad():
        for i, (X, y) in enumerate(progress_bar):
            X, y = X.to(device), y.to(device)
            
            pred_logits = model(X)
            pred_labels = torch.argmax(pred_logits, dim=1)
            pred_labels = pred_labels.cpu().numpy().flatten()
            
            preds.append(pred_labels)
            labels.append(y.cpu().numpy().flatten())
            
            cm = confusion_matrix(labels[-1], preds[-1], labels=range(num_classes))

            TP = np.diag(cm)
            FP = cm.sum(axis=0) - TP
            FN = cm.sum(axis=1) - TP

            iou_per_class = np.divide(TP, TP + FP + FN, out=np.zeros_like(TP, dtype=float), where=(TP + FP + FN) != 0)

            union = TP + FP + FN

            mean_iou_img = np.mean(iou_per_class[union > 0])
            
            image_ious.append((i, mean_iou_img))

            full_cm += cm

            progress_bar.set_postfix(mIoU=mean_iou_img)
    
    sorted_images = sorted(image_ious, key=lambda item: item[1])

    TP = np.diag(full_cm)
    FP = full_cm.sum(axis=0) - TP
    FN = full_cm.sum(axis=1) - TP
    TN = full_cm.sum() - (TP + FP + FN)

    class_precision = np.divide(TP, TP + FP, out=np.zeros_like(TP, dtype=float), where=(TP + FP) != 0)
    #class_acc = np.divide(TP + TN, TP + TN + FP + FN + 1, out=np.zeros_like(TP, dtype=float))
    class_iou = np.divide(TP, TP + FP + FN, out=np.zeros_like(TP, dtype=float), where=(TP + FP + FN) != 0)

    metrics_df = pd.DataFrame({
        "Classe": class_names,
        #'Acurácia': class_acc,
        "Precisão": class_precision,
        "IoU": class_iou
    })

    sorted_metrics_df = metrics_df.sort_values(by="IoU", ascending=True)
    print(sorted_metrics_df.to_string())
    
    return sorted_metrics_df, cm, sorted_images

classes_list = classes["name"].tolist()
num_classes = len(classes_list)

metrics_df, confusion_m, worst_imgs = model_eval(val_dataloader, model, num_classes, classes_list)

# Mapa de Erro

In [ ]:
def plot_error_map(dataset, modelo, image_index, colors_tensor):
    modelo.eval()
    
    img_tensor, label_tensor = dataset[image_index]
    img_tensor = img_tensor.cpu()
    label_tensor = label_tensor.cpu()

    with torch.no_grad():
        pred_logits = modelo(img_tensor.unsqueeze(0).to(device))
        pred_tensor = torch.argmax(pred_logits, dim=1).squeeze(0).cpu()

    gt_mask_np = colors_tensor[label_tensor.long()].numpy().astype(np.uint8)
    pred_mask_np = colors_tensor[pred_tensor.long()].numpy().astype(np.uint8)

    mean = torch.tensor([105.3549, 107.8312, 109.6686]).view(3, 1, 1)
    std = torch.tensor([5637.0488**0.5, 5869.9844**0.5, 5731.8906**0.5]).view(3, 1, 1)
    
    original_img_tensor = img_tensor * std + mean
    original_img_np = original_img_tensor.byte().permute(1, 2, 0).numpy()

    error_mask = (pred_tensor != label_tensor).numpy()
    error_overlay_np = original_img_np.copy()
    error_overlay_np[error_mask] = [255, 0, 0]

    fig, axes = plt.subplots(1, 4)
    
    axes[0].imshow(original_img_np)
    axes[0].set_title(f"Original - idx: {image_index}")

    axes[1].imshow(gt_mask_np)
    axes[1].set_title("True Mask (GT)")

    axes[2].imshow(pred_mask_np)
    axes[2].set_title("Model Pred")

    axes[3].imshow(error_overlay_np)
    axes[3].set_title("Error")

    for ax in axes:
        ax.axis("off")

    plt.tight_layout()
    plt.show()

colors_tensor = torch.Tensor(classes[["r", "g", "b"]].to_numpy()).to(int)

for img_idx, iou in worst_imgs[:3]:
    plot_error_map(val_data, model, img_idx, colors_tensor)

# Data Augmentation

In [ ]:
import torchvision.transforms.functional as F

class CamVidDatasetAug(CamVidDataset):
    def __init__(self, type_, img_dim=[720, 960], dataset_path=None, device="cpu", aug=True):
        super().__init__(type_, img_dim, dataset_path, device)
        self.use_aug = aug
        
        if self.use_aug:
            self.tilt_prob = 0.5
            self.flip_prob = 0.5
            self.zoom_prob = 0.5
            self.shear_prob = 0

    def __getitem__(self, idx):
        def zoom(image, label, scale1, scale2=1.0):
            i, j, h, w = T.RandomResizedCrop.get_params(
                image, scale=(scale1, scale2), ratio=(1.0, 1.0)
            )
            image = F.resized_crop(image, i, j, h, w, image.shape[1:])
            label = F.resized_crop(label, i, j, h, w, label.shape[1:], interpolation=F.InterpolationMode.NEAREST)
            return image, label

        # Pega o path da imagem e da label e carrega
        img_path = self.data[idx]
        name = os.path.basename(img_path).split('.')[0]
        label_path = os.path.join(self.base_path_labels, f"{name}_L.png")

        # Carrega como tensores na CPU
        image = decode_image(img_path).to(torch.float)
        label = decode_image(label_path)

        # Aplica augmentation (se habilitado)
        if self.use_aug:
            if torch.rand(1) < self.tilt_prob:
                angle = torch.randint(-10, 10, (1,)).item()
                image = F.rotate(image, angle, fill=0)
                label = F.rotate(label, angle, interpolation=F.InterpolationMode.NEAREST, fill=0)
            
            if torch.rand(1) < self.flip_prob:
                image = F.hflip(image)
                label = F.hflip(label)
                
            if torch.rand(1) < self.zoom_prob:
                image, label = zoom(image, label, 0.5)

                

        # Aplica transformações de pré-processamento
        image = self.resize_img(image)
        image = self.normalize_img(image)
        label = self.resize_label(label)

        # Converte a máscara RGB para classes
        label_permute = label.permute(1, 2, 0)
        r_channel = label_permute[:, :, 0]
        g_channel = label_permute[:, :, 1]
        b_channel = label_permute[:, :, 2]
        label = torch.from_numpy(self.lookup_classes[r_channel, g_channel, b_channel])

        # Move os tensores finais para o device correto
        return image.to(self.device), label.to(self.device)

In [ ]:
def test_hflip_augmentation(dataset_class, dataset_path, device, colors_tensor, image_index=0):
    """
    Carrega a mesma imagem várias vezes de um dataset com augmentation
    para verificar visualmente se a transformação (hflip) está sendo aplicada
    corretamente tanto na imagem quanto na máscara.
    """
    print(f"Testando hflip para a imagem de índice {image_index}. Execute novamente para ver diferentes resultados aleatórios.")
    
    # Instancia o dataset com augmentation ativada
    aug_dataset = dataset_class(
        "train", 
        img_dim=[256, 256], 
        dataset_path=[dataset_path], 
        device=device, 
        aug=True
    )
    
    # Define os parâmetros de desnormalização para visualização
    mean = torch.tensor([105.3549, 107.8312, 109.6686], device=device).view(3, 1, 1)
    std = torch.tensor([5637.0488**0.5, 5869.9844**0.5, 5731.8906**0.5], device=device).view(3, 1, 1)
    
    # Plota a mesma imagem 4 vezes para observar o efeito aleatório do flip
    fig, axes = plt.subplots(4, 2, figsize=(8, 16))
    fig.suptitle("Verificação do Random Horizontal Flip (hflip)", fontsize=16)

    for i in range(4):
        # Pega a imagem e a máscara do dataset
        img_tensor, mask_tensor = aug_dataset[image_index]
        
        # 1. Prepara a imagem para visualização (desnormaliza)
        img_display = img_tensor * std + mean
        img_display = img_display.byte().cpu().permute(1, 2, 0).numpy()
        
        # 2. Prepara a máscara para visualização (converte de classe para cor)
        mask_display = colors_tensor[mask_tensor.long()].cpu().numpy().astype(np.uint8)
        
        # Plota a imagem
        axes[i, 0].imshow(img_display)
        axes[i, 0].set_title(f"Tentativa {i+1}: Imagem")
        axes[i, 0].axis('off')
        
        # Plota a máscara
        axes[i, 1].imshow(mask_display)
        axes[i, 1].set_title(f"Tentativa {i+1}: Máscara")
        axes[i, 1].axis('off')
        
    plt.tight_layout(rect=[0, 0.03, 1, 0.96])
    plt.show()

# --- Célula para executar o teste ---
# Certifique-se que as variáveis 'camvid_path', 'device' e 'classes' já foram definidas em células anteriores.
colors_tensor = torch.Tensor(classes[["r", "g", "b"]].to_numpy()).to(int)

# Execute a função de teste
test_hflip_augmentation(CamVidDatasetAug, camvid_path, device, colors_tensor, image_index=10)

# Plot do histórico e comparações

In [ ]:
def plot_hist(hist, name = ""):
    fig, axes = plt.subplots(1, 2, figsize=(10, 4))
    fig.suptitle(f"Histórico de treinamento - {name}")

    axes[0].plot(hist["train"]["loss"], label = "Train")
    axes[0].plot(hist["val"]["loss"], label = "Validation")
    axes[0].set_xlabel("Epoch")
    axes[0].set_ylabel("Loss de treino")
#    axes[0].set_title("Comparação de perda no treino")
    axes[0].legend()

    axes[1].plot(hist["train"]["acc"], label = "Train")
    axes[1].plot(hist["val"]["acc"], label = "Validation")
    axes[1].set_xlabel("Epoch")
    axes[1].set_ylabel("Acurácia")
    axes[1].set_ylim((0, 1))
#    axes[1].set_title("Comparação de acurácia no treino")
    axes[1].legend()

In [ ]:
def compare_acc(hist_1, hist_2, name_1 = "", name_2 = ""):
    plt.plot(hist_1["val"]["acc"], label = name_1)
    plt.plot(hist_2["val"]["acc"], label = name_2)
    plt.xlabel("Epoch")
    plt.ylabel("Acurácia")
    plt.ylim((0, 1))
    plt.title(f"Comparação de acurácias - {name_1} vs {name_2}")
    plt.legend()

# DeepLab V3

In [ ]:
class DeepLabV3(nn.Module):
    def __init__(self, weights = DeepLabV3_ResNet50_Weights.DEFAULT):
        super().__init__()
        self.deeplab = deeplabv3_resnet50(weights = weights)

        for param in self.deeplab.parameters():
            param.requires_grad = False

        self.deeplab.classifier = DeepLabHead(2048, 32)
        self.deeplab.aux_classifier = FCNHead(1024, 32)
    
    def forward(self, x):
        return self.deeplab(x)